In [8]:
!pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets.
train_data = datasets.FashionMNIST(
    root="dataset",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="dataset",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [17]:
# Number of samples:
print(f"Length of train data: {len(train_data)}")
print(f"Length of test data: {len(test_data)}")

Length of train data: 60000
Length of test data: 10000


In [4]:
BATCH_SIZE = 64

# Create dataloaders.
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")  # (Batch Size, Channels, Height, Width)
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

[2022-03-19 08:23:09.197 1-8-1-cpu-py36-ml-t3-medium-62c6b413a5e8d67a1da6b0c48d04:34 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-03-19 08:23:09.476 1-8-1-cpu-py36-ml-t3-medium-62c6b413a5e8d67a1da6b0c48d04:34 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [20]:
# Number of iterations: 
print(f"Length of train dataloader: {len(train_dataloader)}")  # 60000 / 64 = 938
print(f"Length of test dataloader: {len(test_dataloader)}")    # 10000 / 64 = 157

Length of train dataloader: 938
Length of test dataloader: 157


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
from torchsummary import summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Total number of samples in train set.
    model.train()
    for batch, (X, y) in enumerate(dataloader):  # Iterations.
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error.
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        
        # Backpropagation.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)  # Iteration * Batch Size = Number of samples have being trained.
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)  # Total number of samples in test set.
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294084  [    0/60000]
loss: 2.287070  [ 6400/60000]
loss: 2.266164  [12800/60000]
loss: 2.270161  [19200/60000]
loss: 2.254373  [25600/60000]
loss: 2.225049  [32000/60000]
loss: 2.239859  [38400/60000]
loss: 2.202188  [44800/60000]
loss: 2.195063  [51200/60000]
loss: 2.170850  [57600/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.162962 

Epoch 2
-------------------------------
loss: 2.165450  [    0/60000]
loss: 2.161936  [ 6400/60000]
loss: 2.104070  [12800/60000]
loss: 2.129931  [19200/60000]
loss: 2.093546  [25600/60000]
loss: 2.025709  [32000/60000]
loss: 2.060524  [38400/60000]
loss: 1.978693  [44800/60000]
loss: 1.974362  [51200/60000]
loss: 1.916236  [57600/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.909250 

Epoch 3
-------------------------------
loss: 1.932511  [    0/60000]
loss: 1.909522  [ 6400/60000]
loss: 1.792458  [12800/60000]
loss: 1.840992  [19200/60000]
loss: 1.754990  [25600/60000]
loss: 1.686986  [32000/600